# Run these after removing hastag if any on of the given module is missing 

In [ ]:
#remove hastag if having problem with gensim
import sys

#!$sys.executable -m pip install keras
#!$sys.executable -m pip install nltk
import nltk

#!$sys.executable -m pip install gensim

#nltk.download('punkt')

#nltk.download('stopwords')

#!$sys.executable -m pip install tensorflow

# Data loading and cleaning

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
DATASET_DIR="Dataset and discription/"

In [ ]:
X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')

In [ ]:
X.head()

In [ ]:
X=X.dropna(axis=1)

In [ ]:
X=X.drop(columns=['rater1_domain1','rater2_domain1'])


In [ ]:
X

In [ ]:
Y=X['domain1_score']

In [ ]:
minimum_scores = np.array([-1, 2, 1, 0, 0, 0, 0, 0, 0])
maximum_scores = np.array([-1, 12, 6, 3, 3, 4, 4, 30, 60])

In [ ]:
old_min = minimum_scores[X['essay_set']]
old_max = maximum_scores[X['essay_set']]
old_range = old_max - old_min 
new_range = (10 - 0)  
X['score'] = np.around((((X['domain1_score'] - old_min) * new_range) / old_range) )

X



# Tokanization and vectorization functions

In [ ]:
from nltk.corpus import stopwords
import nltk

import re
def wordlist(essay, remove_stopwords):
    
    essay = re.sub("[^a-zA-Z]", " ", essay)
    words = essay.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

In [ ]:
def Make_sentences(essay, remove_stopwords):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(wordlist(raw_sentence, remove_stopwords))
    return sentences

In [ ]:
def makeFeatureVec(words, model, num_features):
    
    featureVec = np.zeros((num_features),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

    

In [ ]:
def getAvgFeatureVecs(essays, model, num_features):

    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

# Defining production model

In [ ]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

# Example of pre processing , tokanization then vectorivation

# Example of tokanization

In [ ]:
from gensim.models import Word2Vec
# define training data
example2='hi I am an example1 sentence here to explain thw working of above function ,DONT MIND THE ENGLISH i AM REALLT BAD AT ENGLISH'
example1='hi I am an example22 sentence here to explain thw working of above function ,I WOULD REALLY APPRECIATE FEEDBACKS '
example3='hi I am an example33 sentence here to explain thw working of above function ,I am really thanksful for the helps online'
s=[]
s +=Make_sentences(example1,remove_stopwords=True)
s +=Make_sentences(example2,remove_stopwords=True)
s +=Make_sentences(example3,remove_stopwords=True)
print(s)


# Example of vectorization

In [ ]:
model = Word2Vec(s, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.index_to_key )
print(words)
# access vector for one word
print(model.wv['hi'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

# Onto the real model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import nltk
from gensim.models import Word2Vec
skf = StratifiedKFold(n_splits=5,shuffle=True)
count=1
results=[]
for train,test in skf.split(X,Y):
        print("\n--------Fold {}--------\n".format(count))
        X_test, X_train, y_test, y_train = X.iloc[test], X.iloc[train], Y.iloc[test], Y.iloc[train]
        trainE = X_train['essay']
        testE= X_test['essay']
        sentences=[]
        for essay in trainE:
        # Obtaining all sentences from the training essays.
        
            sentences +=Make_sentences(essay, remove_stopwords = True)
        
        #Initializing different parameters for the word2vec model to be used
    
        num_features = 300
        min_word_count = 40
        num_workers = 4
        context = 10
        downsampling = 1e-3

        print("Training Word2Vec Model...")
        
        #Initializing model fro vectorization
        
    
        #initializing model and loading parameters
    
        model = Word2Vec(sentences,vector_size=300, workers=num_workers, min_count = min_word_count, window = context)
        #avoiding normalization to not reduce the essence of some words used in context
    
        model.init_sims(replace=False)
    
        #saving model
    
        model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
        
        print("--------------- Traning complete-------------------")
        
        print("--------------- Initializing vectorization on train set -------------------")
        
        
        #generating vectors train
    
        clean_train = []
        for essay in trainE:
            clean_train.append(wordlist(essay, remove_stopwords=True))
        trainDataVecs = getAvgFeatureVecs(clean_train, model, num_features)
    
        print("--------------completed-----------------")
    
        print("--------------- Initializing vectorization on test set -------------------")
        
        #generating vectors for test
    
        clean_test = []
        for essay in testE:
            clean_test.append(wordlist(essay, remove_stopwords=True))
        testDataVecs = getAvgFeatureVecs(clean_test, model, num_features)
        
        trainDataVecs = np.array(trainDataVecs)
        testDataVecs = np.array(testDataVecs)
        
        print("------------Complete----------------------") 
        
        print("------------Adding extra dimension to essay----------------------") 
        
        #Adding extra dimension to essay
        
        trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
        
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
        
        print("------------Complete----------------------") 
        
        print("------------Traning the Preprocessed Data----------------------") 
        
        #Traning the Preprocessed Data
        
        lstm_model = get_model()
        
        print('-----------Complete--------------')
        
        print('------------------fitting data----------------------')
        
        #fitting dataset
        checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
        lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50,callbacks=[checkpoint],validation_split=0.1)
        
        print('-----------Complete--------------')
        
        print('---------------Predicting test set------------------------')
        
        #predicting test
        
        y_pred = lstm_model.predict(testDataVecs)
    
        # Save any one of the 8 models.
        
        if count == 5:
             lstm_model.save('./final_lstm.h5')
            
        # Round y_pred to the nearest integer.
        y_pred = np.around(y_pred)
    
            # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
        result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
        print("Kappa Score: {}".format(result))
        results.append(result)

        count += 1


**The traning of the model is complete ****